# Pose Estimation using YOLOv7 Algorithm!

Previously, I worked on several computer vision and YOLOv3 projects like the automatic out-of-stock inventory management system. But, since YOLOv7 is considered as one of the best object detection algorithms, I gave it a try! Created in the year 2015, by Joseph Redmon (who also proposed YOLOv2 & YOLOv3). Since then, the open soure contributors have collaborated together to create more versions in the YOLO family like:

YOLOv4, YOLOv5, PP-YOLO, Scaled YOLOv4, PP-YOLOv2, YOLOv5, YOLOv6, and YOLOv7 (built on top of YOLOR - You Only Learn One Representation). YOLOv7 is more than just an object detection architecture. It provides a new model head that emits keypoints (skeleton) and can perform instance segmentation with just bounding box regression.

Why did I use a YOLO model?
1. Enables me to process video feeds at a high frames-per-second rate. My computer uses intel integrated graphics, which poses various processing limitations. 

2. Continuing R&D from the open-source community. YOLOv7 being the latest addition. 

3. No shortage of information or bug removals while implementing a model.

# 

## Downloading Necessary Weights
Convolutional Neural Network (CNN) like the YOLO need a lot of images with variations to train on. To avoid redundancy, we transfer the learnings of the first few layers and just needs to learn the last (or maybe last few) layers to work for your specific use case.

In [ ]:
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt -o yolov7-w6-pose.pt

# 

## Importing Libraries

In [2]:
#For common image transformations
import torch
from torchvision import transforms

#Selects a subset of bounding boxes in descending order of score
from utils.general import non_max_suppression_kpt

#To resize and pad the video to a shape that the model can work with
from utils.datasets import letterbox
from utils.plots import output_to_keypoint, plot_skeleton_kpts

#for graphs & plots
import matplotlib.pyplot as plt

#for image and video analysis 
import cv2

#for working with arrays
import numpy as np

# 

## CPU or GPU availability check. Then, load pre-trained model
Having a GPU quickens the processing as it switches from float32 to float16.

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_model():
    model = torch.load('Libraries/yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

# 

## Transform Video & Run through the model!

In [4]:
def run_inference(image):
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (567, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 567, 960])
    if torch.cuda.is_available():
        image = image.half().to(device)
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 567, 960])
    with torch.no_grad():
        output, _ = model(image)
    return output, image

# 

## Returning Predictions & Image as a Tensor

In [5]:
def draw_keypoints(output, image):
    output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)

    return nimg

# 

## Using OpenCV to read a video:
We run this entire process for every frame on a video. On each frame, we'll also write the frame into a new file, encoded as a video. This process will take significant time (more if done without a GPU).

In [6]:
def pose_estimation_video(filename):
    cap = cv2.VideoCapture(filename)
    # VideoWriter for saving the video
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('Video_output.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))
    while cap.isOpened():
        (ret, frame) = cap.read()
        if ret == True:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            output, frame = run_inference(frame)
            frame = draw_keypoints(output, frame)
            frame = cv2.resize(frame, (int(cap.get(3)), int(cap.get(4))))
            out.write(frame)
            cv2.imshow('Pose estimation', frame)
        else:
            break

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()


# 

## Final Deployment

In [ ]:
pose_estimation_video('ice_skating.mp4')

C:\Users\Dr Juned bhai\Conda\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# 

## Then, the newly generated gets saved as 'Video_output' in the same directory! The last process will take some time, so be patient!

# 

# Thank you!